# Evaluate DeepFace's Results In The Big Picture

In [1]:
import pandas as pd
from IPython.display import display, HTML
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
alignment = [False, True]
models = ["Facenet512", "Facenet", "VGG-Face", "ArcFace", "Dlib", "GhostFaceNet", "SFace", "OpenFace", "DeepFace", "DeepID"]
detectors = ["retinaface", "mtcnn", "fastmtcnn", "dlib", "yolov8", "yunet", "centerface", "mediapipe", "ssd", "opencv", "skip"]
distance_metrics = ["euclidean", "euclidean_l2", "cosine"]

# Main results

In [3]:
for align in alignment:
    for metric in distance_metrics:
        df = pd.read_csv(f"results/pivot_{metric}_with_alignment_{align}.csv")
        df = df.rename(columns = {'Unnamed: 0': 'detector'})
        df = df.set_index('detector')

        print(f"{metric} for alignment {align}")
        display(HTML(df.to_html()))
        display(HTML("<hr>"))

FileNotFoundError: [Errno 2] No such file or directory: 'results/pivot_euclidean_with_alignment_False.csv'

In [ ]:
def create_github_table():
    for metric in distance_metrics:
        for align in [True, False]:
            df = pd.read_csv(f"results/pivot_{metric}_with_alignment_{align}.csv")
            df = df.rename(columns = {'Unnamed: 0': 'detector'})
            df = df.set_index('detector')

            print(f"Performance Matrix for {metric} while alignment is {align} \n")
            header = "| | "
            for col_name in df.columns.tolist():
                header += f"{col_name} |"
            print(header)
            # -------------------------------
            seperator = "| --- | "
            for col_name in df.columns.tolist():
                seperator += " --- |"
            print(seperator)
            # -------------------------------
            for index, instance in df.iterrows():
                line = f"| {instance.name} |"
                for i in instance.values:
                    if i < 97.5:
                        line += f"{i} |"
                    else:
                        line += f"**{i}** |"
                print(line)

            print("\n---------------------------")

In [ ]:
# create_github_table()

# Alignment impact

In [ ]:
align_df = None

for distance_metric in distance_metrics:
    df1 = (
        pd.read_csv(f"results/pivot_{distance_metric}_with_alignment_True.csv")
        .rename(columns = {'Unnamed: 0': 'detector'})
        .set_index('detector')
    )
    df2 = (
        pd.read_csv(f"results/pivot_{distance_metric}_with_alignment_False.csv")
        .rename(columns = {'Unnamed: 0': 'detector'})
        .set_index('detector')
    )
    df1 = df1[df1.index != "skip"]
    df2 = df2[df2.index != "skip"]
    pivot_df = df1.subtract(df2)

    pivot_df = pivot_df.max()
    pivot_df = pd.DataFrame(pivot_df, columns=[f'alignment_impact_of_{distance_metric}'])
    # display(HTML(pivot_df.to_html()))

    if align_df is None:
        align_df = pivot_df.copy()
    else:
        align_df = align_df.merge(pivot_df, left_index=True, right_index=True)

# display(HTML(align_df.to_html()))
align_df = pd.DataFrame(align_df.max(axis=1), columns = ["max_alignment_impact"])
align_df = align_df.sort_values(by=["max_alignment_impact"], ascending=False)
display(HTML(align_df.to_html()))

## Detection impact

In [ ]:
detect_df = None
for distance_metric in distance_metrics:
    tmp_df = (
        pd.read_csv(f"results/pivot_{distance_metric}_with_alignment_False.csv")
        .rename(columns = {'Unnamed: 0': 'detector'})
        .set_index('detector')
    )
    ref_df = tmp_df[tmp_df.index == "skip"]

    j = []
    for i in range(0, len(detectors) - 1):
        j.append(ref_df)
    minus_df = pd.concat(j)

    tmp_df = tmp_df[tmp_df.index != "skip"]
    minus_df.index = tmp_df.index

    # print("performance with no detection")
    # display(HTML(ref_df.to_html()))

    # print("pivot")
    tmp_df = tmp_df.subtract(minus_df)
    # display(HTML(tmp_df.to_html()))

    # print("avg of detector impact for models")
    # avg_df = pd.DataFrame(tmp_df.mean()).T
    avg_df = pd.DataFrame(tmp_df.max(), columns=[f"detection_impact_of_{distance_metric}"])
    # display(HTML(avg_df.to_html()))

    if detect_df is None:
        detect_df = avg_df.copy()
    else:
        detect_df = detect_df.merge(avg_df, left_index=True, right_index=True)

# display(HTML(detect_df.to_html()))
detect_df = pd.DataFrame(detect_df.max(axis=1), columns = ["max_detection_impact"])
detect_df = detect_df.sort_values(by=["max_detection_impact"], ascending=False)
display(HTML(detect_df.to_html()))


# facial recognition model's best scores

In [ ]:
df = pd.DataFrame()
for align in alignment:
    for distance_metric in distance_metrics:
        tmp_df = (
            pd.read_csv(f"results/pivot_{distance_metric}_with_alignment_{align}.csv")
            .rename(columns = {'Unnamed: 0': 'detector'})
            .set_index('detector')
        )
        df = pd.concat([df, tmp_df])

pivot_df = pd.DataFrame(df.max(), columns = ["best_accuracy_score"])

# add human comparison
pivot_df.loc["Human-beings"] = 97.5

pivot_df = pivot_df.sort_values(by = ["best_accuracy_score"], ascending = False)
pivot_df

# ROC Curves

In [ ]:
def plot_roc(model_name, detector_backend, distance_metric, align):
    alignment_text = "aligned" if align == True else "unaligned"

    df = pd.read_csv(f"outputs/test/{model_name}_{detector_backend}_{distance_metric}_{alignment_text}.csv")

    #normalize
    df["distances_normalized"] = df["distances"] / df["distances"].max()
    df["actuals_normalized"] = 0
    idx = df[df["actuals"] == False].index
    df.loc[idx, "actuals_normalized"] = 1

    y_actual = df["actuals_normalized"].values.tolist()
    y_pred_proba = df["distances_normalized"].values.tolist()

    fpr, tpr, _ = metrics.roc_curve(y_actual, y_pred_proba)
    auc = metrics.roc_auc_score(y_actual, y_pred_proba)
    auc = round(auc, 4)

    # best accuracy score
    result_path = f"results/pivot_{distance_metric}_with_alignment_{align}.csv"
    result_df = pd.read_csv(result_path)
    acc = result_df[result_df["Unnamed: 0"] == detector_backend][model_name].values[0]

    label = f"{model_name}_{detector_backend}_{distance_metric}_{alignment_text} (acc: {acc}, auc: {auc})"

    return acc, auc, fpr, tpr, label

In [ ]:
# to show all models in same graph
plt.figure(figsize=(17, 8))

for model_name in models:
    # to show graphs model by model
    # plt.figure(figsize=(17, 8))
    accs = []
    aucs = []
    fprs = []
    tprs = []
    labels = []
    for distance_metric in distance_metrics:
        # for detector_backend in robust_face_detectors:
        for detector_backend in detectors:
            for align in alignment:
                if detector_backend == "skip" and align is True:
                    continue
                acc, auc, fpr, tpr, label = plot_roc(model_name, detector_backend, distance_metric, align)
                accs.append(acc)
                aucs.append(auc)
                fprs.append(fpr)
                tprs.append(tpr)
                labels.append(label)
    # ---------------------------------
    #sort by auc
    df = pd.DataFrame({"acc": accs, "auc": aucs, "fpr": fprs, "tpr": tprs, "label": labels})
    # df = df.sort_values(by = ["auc"], ascending = False).reset_index()
    df = df.sort_values(by = ["acc"], ascending = False).reset_index()

    for index, instance in df.iterrows():
        fpr = instance["fpr"]
        tpr = instance["tpr"]
        auc = instance["auc"]
        acc = instance["acc"]
        label = instance["label"]

        plt.plot(fpr, tpr, label=label)
        plt.ylabel("True Positive Rate")
        plt.xlabel("False Positive Rate")
        plt.legend(loc="lower center", ncol=2)
        # normally this should be [0, 1] but that scale makes graphs not legible
        # plt.xlim([0, 1])
        plt.xlim([0, 0.3])

        # to show the best auc value
        break

    # to show graphs model by model
    # plt.show()
    # print("----------------")

# to show all models in same graph
plt.show()